In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
import argparse
import torch
import json
from unilatent import UniLatentPipeline

from data.builder import build_dataset, build_dataloader
from aspect_ratio_sampler import AspectRatioBatchSampler
from torch.utils.data import RandomSampler

from tqdm import tqdm
from accelerate import Accelerator

/home/tiger/.local/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/home/tiger/.local/lib/python3.9/site-packages/bytedmetrics/__init__.py:10: UserWarning: bytedmetrics is renamed to bytedance.metrics, please using `bytedance.metrics` instead of `bytedmetrics`
  warnings.warn("bytedmetrics is renamed to bytedance.metrics, please using `bytedance.metrics` instead of `bytedmetrics`")


In [7]:
# data_config = {
#     'type': 'FlexibleInternalDataMS',
#     'roots': [
#         # '/mnt/bn/us-aigc-temp/henry/coco_2014/val/val2014/',
#         '/mnt/bn/aigc-us/zjl/laion-coco-aesthetic/data_max1024/',
#     ],
#     'json_lst': [
#         # '/mnt/bn/us-aigc-temp/henry/test.json',
#         '/mnt/bn/aigc-us/zjl/laion-coco-aesthetic/data_max1024/meta_data_coco_edited.json',
#     ],
#     'load_vae_feat': False,
#     'load_t5_feat': False
# }
# dataset = build_dataset(
#     data_config, resolution=512, aspect_ratio_type='ASPECT_RATIO_512',
#     real_prompt_ratio=0.0, max_length=77, return_image_id=True
# )
# batch_sampler = AspectRatioBatchSampler(sampler=RandomSampler(dataset), dataset=dataset,
#                                     batch_size=1, aspect_ratios=dataset.aspect_ratio, drop_last=True,
#                                     ratio_nums=dataset.ratio_nums, valid_num=0)
# dataloader = build_dataloader(dataset, batch_sampler=batch_sampler, num_workers=10)

accelerator = Accelerator(
        mixed_precision='fp16',
    )

data_config = {
    'type': 'FlexibleInternalDataMS',
    'roots': [
        '/mnt/bn/aigc-us/zjl/laion-coco-aesthetic/data_max1024/',
    ],
    'json_lst': [
        '/mnt/bn/aigc-us/zjl/laion-coco-aesthetic/data_max1024/meta_data_coco_edited.json',
    ],
    'load_vae_feat': False,
    'load_t5_feat': False
}
dataset = build_dataset(
    data_config, resolution=512, aspect_ratio_type='ASPECT_RATIO_512',
    real_prompt_ratio=0.0, max_length=77,
)
batch_sampler = AspectRatioBatchSampler(sampler=RandomSampler(dataset), dataset=dataset,
                                    batch_size=1, aspect_ratios=dataset.aspect_ratio, drop_last=True,
                                    ratio_nums=dataset.ratio_nums, valid_num=0)
dataloader = build_dataloader(dataset, batch_sampler=batch_sampler, num_workers=10)

Detected kernel version 5.4.143, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Constructing dataset FlexibleInternalDataMS...


2024-07-27 06:14:56,467 - PixArt - WARNING - Using valid_num=0 in config file. Available 40 aspect_ratios: ['0.25', '0.26', '0.27', '0.28', '0.32', '0.33', '0.35', '0.4', '0.42', '0.48', '0.5', '0.52', '0.57', '0.6', '0.68', '0.72', '0.78', '0.82', '0.88', '0.94', '1.0', '1.07', '1.13', '1.21', '1.29', '1.38', '1.46', '1.67', '1.75', '2.0', '2.09', '2.4', '2.5', '2.89', '3.0', '3.11', '3.62', '3.75', '3.88', '4.0']


Dataset FlexibleInternalDataMS constructed. time: 42.67 s, length (use/ori): 7591625/7596238


In [13]:
def prepare(accelerator, pipe):
    (
        pipe.transformer,
        pipe.text_encoder, 
        pipe.text_encoder_2,
        pipe.clip_image_encoder,
        pipe.text_decoder,
        pipe.vae
    ) = accelerator.prepare(
        pipe.transformer,
        pipe.text_encoder, 
        pipe.text_encoder_2,
        pipe.clip_image_encoder,
        pipe.text_decoder,
        pipe.vae
    )

    return pipe

def dift_sampler(batch, pipe, index, block_num):
    index_ = torch.zeros(size=(1,), dtype=torch.long) + index
    embeds, pooled_embeds = pipe.dift_features(batch[0][:1], index_, return_layer=block_num)
    embeds = torch.cat([embeds, pooled_embeds], axis=1)
    decoded_tokens = pipe.text_decoder.generate_captions(embeds, 
                        eos_token_id=pipe.decoder_tokenizer.eos_token_id, device=accelerator.device)[0]
    decoded_text = pipe.decoder_tokenizer.batch_decode(decoded_tokens)[0]
    return decoded_text

def clip_sampler(batch, pipe):
    embeds, pooled_embeds = pipe.encode_image(batch[0][:1])
    embeds = torch.cat([embeds, pooled_embeds], axis=1)
    decoded_tokens = pipe.text_decoder.generate_captions(embeds, 
                        eos_token_id=pipe.decoder_tokenizer.eos_token_id, device=accelerator.device)[0]
    decoded_text = pipe.decoder_tokenizer.batch_decode(decoded_tokens)[0]
    return decoded_text

def generate_captions(pipe, dataloader, save_path, sampler, sampler_kwargs={}):
    json_list = []
    progbar = tqdm(dataloader)
    for i, batch in enumerate(progbar):
        with torch.no_grad():
            decoded_text = sampler(batch, pipe, **sampler_kwargs)
        
        caption = decoded_text.strip('!').replace('<|endoftext|>', '').replace('<|EOS|>', '').strip()
        image_id = batch[-1]['image_id'].item() if 'image_id' in batch[-1] else 0
        json_list.append({'image_id': image_id, 'caption': caption})

        progbar.set_description(f"Image: {i:05d} | Predicted: {caption} | True: {batch[1][0]}")

        if (i + 1) % 50 == 0:
            with open(save_path, 'w') as f:
                test = json.dump(json_list, f)

    return json_list

In [16]:
epoch = 49999
for block_num in [6]:
    # for index in [0, 250, 500, 750]:
    for index in [500, 750, 250, 0]:
        name = f'index_{index:03d}_block_{block_num}'
        save_path = f'/mnt/bn/us-aigc-temp/henry/data/captions/dift/dift_{name}_step_{epoch}.json'
        load_path = f'/mnt/bn/us-aigc-temp/henry/data/dift/{name}/epoch_0_step_{epoch}/'
        print(f"Loading pipeline from {load_path}")
        pipe = UniLatentPipeline.from_pretrained(load_path, torch_dtype=torch.float32)
        assert torch.allclose(pipe.text_decoder.transformer.lm_head.weight, pipe.text_decoder.transformer.transformer.wte.weight)

        pipe = prepare(accelerator, pipe)
        print(f"Running sampler for {name}:")
        sampler_kwargs = {'index': index, 'block_num': block_num}
        generate_captions(pipe, dataloader, save_path, dift_sampler, sampler_kwargs)

Loading pipeline from /mnt/bn/us-aigc-temp/henry/data/dift/index_500_block_6/epoch_0_step_49999/


Loading pipeline components...:   0%|          | 0/11 [00:00<?, ?it/s]

Some weights of TextDecoder were not initialized from the model checkpoint at /mnt/bn/us-aigc-temp/henry/data/dift/index_500_block_6/epoch_0_step_49999/text_decoder and are newly initialized: ['pooled_image_embedder.weight', 'transformer.lm_head.weight', 'image_embedder.bias', 'image_embedder.weight', 'pooled_image_embedder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running sampler for index_500_block_6:


  0%|          | 0/7591625 [00:00<?, ?it/s]

...

WHOA INSIDE CAPTION DECODER tensor(2815.9836, device='cuda:0') tensor(0.0249, device='cuda:0') tensor(4.4906, device='cuda:0')


Image: 00028 | Predicted: a man in a suit and tie standing outside. | True: an image of the beautiful actress in yellow:   0%|          | 29/7591625 [00:34<2087:57:08,  1.01it/s]     

WHOA INSIDE CAPTION DECODER tensor(2805.9497, device='cuda:0') tensor(0.0529, device='cuda:0') tensor(4.4744, device='cuda:0')


Image: 00029 | Predicted: a man in a white shirt and black pants. | True: A glass table with metal legs and a round top.:   0%|          | 30/7591625 [00:36<2539:50:53,  1.20s/it]


KeyboardInterrupt: 